In [0]:
#%run ./_paramgplayscrapper

In [0]:
#%run ./_envsettings

In [0]:
from pyspark.sql import functions as F

In [0]:
spark.conf.set(f"fs.azure.account.key.{account_name}.dfs.core.windows.net", account_key)

In [0]:
#Writing data to parquest file in silver layer
if keyword_extraction:
    csv_path  = f"abfss://root@{account_name}.dfs.core.windows.net/{keyword_extraction_output_file}"
    df = spark.read.format("csv").option("header", "true").option("multiLine", "true").option("escape", '"').option("quote", '"').schema(csv_keyword_schema).load(csv_path)
    df = df.withColumn("etl_timestamp", F.date_format(F.from_utc_timestamp(F.current_timestamp(), "Asia/Kolkata"),"yyyy-MM-dd HH:mm:ss"))
    display(df)
    delta_path = f"abfss://root@{account_name}.dfs.core.windows.net/datalake/silver/{output_location}"
    df.write.format("delta").mode("append").save(delta_path)

In [0]:
#Writing data to parquest file in silver layer
if sentiment_analyzer:
    csv_path_sentiment  = f"abfss://root@{account_name}.dfs.core.windows.net/{sentiment_analyzer_output_file}"
    df_sentiment = spark.read.format("csv").option("header", "true").option("multiLine", "true").option("escape", '"').option("quote", '"').schema(csv_schema).load(csv_path_sentiment)
    df_sentiment_1 = ( df_sentiment.withColumn("date_proper", F.date_format(F.col("Date"), "yyyy-MM-dd HH:mm:ss")))
    df_sentiment_2 = df_sentiment_1.withColumn("etl_timestamp", F.date_format(F.from_utc_timestamp(F.current_timestamp(), "Asia/Kolkata"),"yyyy-MM-dd HH:mm:ss"))
    #display(df_sentiment_2)
    df_sentiment_final = df_sentiment_2.selectExpr("Review","date_proper as Date","Rating","Document_ID","Scores","Sentiment","etl_timestamp")
    #display(df_sentiment_final)
    delta_path_sentiment = f"abfss://root@{account_name}.dfs.core.windows.net/datalake/silver/{output_location_sentiment_analyzer}"
    df_sentiment_final.write.format("delta").mode("append").save(delta_path_sentiment)

In [0]:
# Writing data to parquet file in silver layer
if topic_modelling:
    csv_path_topic = f"abfss://root@{account_name}.dfs.core.windows.net/{topic_modelling_output_file}"
    df_topic = spark.read.format("csv").option("header", "true").option("multiLine", "true").option("escape", '"').option("quote", '"').schema(csv_topic_schema).load(csv_path_topic)
    # Replace spaces and other invalid characters in column names
    for col in df_topic.columns:
        new_col = col.replace(" ", "_")
        if new_col != col:
            df_topic = df_topic.withColumnRenamed(col, new_col)
    display(df_topic)
    df_topic = df_topic.withColumn("etl_timestamp", F.date_format(F.from_utc_timestamp(F.current_timestamp(), "Asia/Kolkata"),"yyyy-MM-dd HH:mm:ss"))
    delta_path_topic = f"abfss://root@{account_name}.dfs.core.windows.net/datalake/silver/{output_location_topic_modelling}"
    df_topic.write.format("delta").mode("append").save(delta_path_topic)